In [ ]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
%cd ..
from Olivia import Olivia
%cd notebooks

c:\Users\arran\Documents\Curso Inteligencia Artificial\Proyecto\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'


c:\Users\arran\Documents\Curso Inteligencia Artificial\Proyecto\olivia_def
c:\Users\arran\Documents\Curso Inteligencia Artificial\Proyecto\olivia_def\testing


In [ ]:
yelp_business = pd.read_parquet("../datasets/yelp_business.parquet")
yelp_reviews = pd.read_parquet("../datasets/yelp_reviews.parquet")
attributes = pd.read_parquet("../datasets/attributes_multihot.parquet")
categories = pd.read_parquet("../datasets/categories_multihot.parquet")
features = pd.read_parquet("../datasets/features.parquet")
reviews_emb = pd.read_parquet("../datasets/restaurant_reviews_pooled_embeddings.parquet").to_numpy()
features_emb = pd.read_parquet("../datasets/features_embeddings.parquet").to_numpy()
with open("../summary_cache.json", "r", encoding="utf-8") as f :
    summary_dict = json.load(f)

In [19]:
inputs = pd.read_csv("testing_inputs.csv")
inputs["initial_features_avg_cossim"] = 0.0
inputs["filtered_features_avg_cossim"] = 0.0
inputs["final_features_avg_cossim"] = 0.0
inputs["initial_reviews_avg_cossim"] = 0.0
inputs["filtered_reviews_avg_cossim"] = 0.0
inputs["final_reviews_avg_cossim"] = 0.0
inputs["top5_prop_matched_attrs"] = 0.0
inputs["avg_prop_matched_attrs"] = 0.0
inputs.head()

,input,included_attributes,initial_features_avg_cossim,filtered_features_avg_cossim,final_features_avg_cossim,initial_reviews_avg_cossim,filtered_reviews_avg_cossim,final_reviews_avg_cossim,top5_prop_matched_attrs,avg_prop_matched_attrs
0,Find me a restaurant serving desserts with fea...,"GoodForMeal.dinner, DietaryRestrictions.gluten...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,What are some good places for salads and pasta...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Can you recommend somewhere with no special fe...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Find me a restaurant serving pizza and thai an...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Suggest a tapas and sushi spot where I can fin...,"HappyHour, WheelchairAccessible, WiFi_no, Good...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
def top_indices(sims, k):
    """Devuelve los índices de los k mayores valores, ordenados de mayor a menor."""
    return np.argsort(sims)[-k:][::-1]

def ri_test_model(inputs, features_emb, reviews_emb, features, first):
    
    model = SentenceTransformer("all-MiniLM-L6-v2")

    for i in tqdm(range(len(inputs))):

        input_emb = model.encode(inputs.loc[i, "input"]).reshape(1, -1)

        feat_sims = cosine_similarity(input_emb, features_emb)[0]
        rev_sims  = cosine_similarity(input_emb, reviews_emb)[0]
        inputs.loc[i, "initial_features_avg_cossim"] = feat_sims.mean()
        inputs.loc[i, "initial_reviews_avg_cossim"]  = rev_sims.mean()

        if first == "reviews":
            top100_idx = top_indices(rev_sims, 100)
        else:
            top100_idx = top_indices(feat_sims, 100)
        
        feat_sims_top100 = feat_sims[top100_idx]
        rev_sims_top100  = rev_sims[top100_idx]
        inputs.loc[i, "filtered_features_avg_cossim"] = feat_sims_top100.mean()
        inputs.loc[i, "filtered_reviews_avg_cossim"]  = rev_sims_top100.mean()

        if first == "reviews":
            top5_rel_idx = top_indices(feat_sims_top100, 5)
        else:
            top5_rel_idx = top_indices(rev_sims_top100, 5)

        top5_idx = top100_idx[top5_rel_idx]

        feat_sims_top5 = feat_sims[top5_idx]
        rev_sims_top5  = rev_sims[top5_idx]
        inputs.loc[i, "final_features_avg_cossim"] = feat_sims_top5.mean()
        inputs.loc[i, "final_reviews_avg_cossim"]  = rev_sims_top5.mean()

        attr_str = inputs.loc[i, "included_attributes"]
        if isinstance(attr_str, str) and attr_str.strip():
            attr_list = [a.strip() for a in attr_str.split(",")]
        else:
            attr_list = []

        if not attr_list:
            inputs.loc[i, "top5_prop_matched_attrs"] = pd.NA
            inputs.loc[i, "avg_prop_matched_attrs"] = pd.NA
            continue

        features_top5 = features.iloc[top5_idx]

        matches = sum(
            1 for attr in attr_list
            if attr in features.columns and features_top5[attr].sum() > 0
        )

        inputs.loc[i, "top5_prop_matched_attrs"] = matches / len(attr_list)

        props = []
        for _, negocio in features_top5.iterrows():
            activos = sum(1 for attr in attr_list if attr in negocio.index and negocio[attr] == 1)
            props.append(activos / len(attr_list))

        inputs.loc[i, "avg_prop_matched_attrs"] = np.mean(props)

    return inputs

In [21]:
inputs_feat_to_rev = ri_test_model(inputs.copy(), features_emb, reviews_emb, features, first = "features")
inputs_feat_to_rev.head()

100%|██████████| 10000/10000 [37:20<00:00,  4.46it/s]


,input,included_attributes,initial_features_avg_cossim,filtered_features_avg_cossim,final_features_avg_cossim,initial_reviews_avg_cossim,filtered_reviews_avg_cossim,final_reviews_avg_cossim,top5_prop_matched_attrs,avg_prop_matched_attrs
0,Find me a restaurant serving desserts with fea...,"GoodForMeal.dinner, DietaryRestrictions.gluten...",0.559691,0.678357,0.687395,0.421052,0.413617,0.524164,0.400000,0.240000
1,What are some good places for salads and pasta...,NaN,0.381741,0.547953,0.563035,0.392185,0.367161,0.497233,NaN,NaN
2,Can you recommend somewhere with no special fe...,NaN,0.403843,0.540597,0.540305,0.392573,0.455784,0.568016,NaN,NaN
3,Find me a restaurant serving pizza and thai an...,NaN,0.440509,0.596868,0.594867,0.406479,0.413284,0.475460,NaN,NaN
4,Suggest a tapas and sushi spot where I can fin...,"HappyHour, WheelchairAccessible, WiFi_no, Good...",0.461215,0.620004,0.615226,0.411723,0.535328,0.590757,0.666667,0.453333


In [22]:
inputs_rev_to_feat = ri_test_model(inputs.copy(), features_emb, reviews_emb, features, first = "reviews")
inputs_rev_to_feat.head()

100%|██████████| 10000/10000 [37:32<00:00,  4.44it/s]


,input,included_attributes,initial_features_avg_cossim,filtered_features_avg_cossim,final_features_avg_cossim,initial_reviews_avg_cossim,filtered_reviews_avg_cossim,final_reviews_avg_cossim,top5_prop_matched_attrs,avg_prop_matched_attrs
0,Find me a restaurant serving desserts with fea...,"GoodForMeal.dinner, DietaryRestrictions.gluten...",0.559691,0.561833,0.639721,0.421052,0.557251,0.556639,0.666667,0.373333
1,What are some good places for salads and pasta...,NaN,0.381741,0.389124,0.489633,0.392185,0.585104,0.581335,NaN,NaN
2,Can you recommend somewhere with no special fe...,NaN,0.403843,0.418424,0.512133,0.392573,0.590706,0.595013,NaN,NaN
3,Find me a restaurant serving pizza and thai an...,NaN,0.440509,0.426318,0.478677,0.406479,0.574897,0.586934,NaN,NaN
4,Suggest a tapas and sushi spot where I can fin...,"HappyHour, WheelchairAccessible, WiFi_no, Good...",0.461215,0.532130,0.617070,0.411723,0.587231,0.588350,0.666667,0.466667


In [24]:
metrics_cols = [
    "initial_features_avg_cossim", "filtered_features_avg_cossim", "final_features_avg_cossim",
    "initial_reviews_avg_cossim", "filtered_reviews_avg_cossim", "final_reviews_avg_cossim",
    "top5_prop_matched_attrs", "avg_prop_matched_attrs"
]
mean_feat_to_rev = inputs_feat_to_rev[metrics_cols].mean()
mean_rev_to_feat = inputs_rev_to_feat[metrics_cols].mean()

# Unir ambas series en un DataFrame
means_combined = pd.DataFrame({
    "feat_to_rev": mean_feat_to_rev,
    "rev_to_feat": mean_rev_to_feat
})
means_combined

,feat_to_rev,rev_to_feat
initial_features_avg_cossim,0.431657,0.431657
filtered_features_avg_cossim,0.580408,0.462177
final_features_avg_cossim,0.581462,0.541321
initial_reviews_avg_cossim,0.362601,0.362601
filtered_reviews_avg_cossim,0.396031,0.547472
final_reviews_avg_cossim,0.497919,0.551302
top5_prop_matched_attrs,0.439269,0.444898
avg_prop_matched_attrs,0.252703,0.269310


In [8]:
means_combined.to_csv("cossims_means.csv", index = False)

In [9]:
olivia = Olivia(yelp_business, yelp_reviews, attributes, categories, summary_dict = summary_dict)
olivia.load_features_embeddings(features_emb)
olivia.load_review_embeddings(reviews_emb)

Device set to use cpu


In [10]:
results = pd.DataFrame(columns=yelp_business.columns)
results["review_summary"] = None
results.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,attributes_text,review_summary


In [11]:
np.random.seed(32)
sample = inputs.sample(20)

sumy_time = []
bart_sum = []
bart_time = []
gemini_sum = []
gemini_time = []


for i in tqdm(range(sample.shape[0])):
    start_time = time.time()
    olivia.reccomend(sample.iloc[i]["input"], target_state="All", summarizer="sumy")
    end_time = time.time()
    sumy_time.append((end_time - start_time) / 5)
    results = pd.concat([results, olivia.get_reccomendations()], ignore_index=True)

for business in tqdm(results["business_id"]):
    start_time = time.time()
    bart_sum.append(olivia.generate_summarizer_bart(yelp_reviews.loc[yelp_reviews["business_id"] == business, "text"]))
    end_time = time.time()
    bart_time.append(end_time - start_time)

    start_time = time.time()
    gemini_sum.append(olivia.generate_summarizer_gemini(
         business,
         yelp_reviews.loc[yelp_reviews["business_id"] == business, "text"].tolist(),
         yelp_reviews.loc[yelp_reviews["business_id"] == business, "stars"].iloc[0]
    ))
    end_time = time.time()
    gemini_time.append(end_time - start_time)

results.rename(columns = {"review_summary" : "sumy"})
results["sumy_time"] = np.repeat(sumy_time, 5)
results["bart"] = bart_sum
results["bart_time"] = bart_time
results["gemini"] = gemini_sum
results["gemini_time"] = gemini_time
results["input"] = sample["input"].repeat(5).reset_index(drop=True)


results.head(10)

  0%|          | 0/100 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 150, but your input_length is only 91. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
 12%|█▏        | 12/100 [07:37<1:09:52, 47.64s/it]Your max_length is set to 150, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
Your max_length is set to 150, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
 30%|███       | 30/10

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,attributes_text,review_summary,text,sumy_time,bart,bart_time,gemini,gemini_time,input
0,_58WfMGlrmnPossd2Gps6g,Black Radish Grocer,2923 N 12th St,Tampa,FL,33605,27.970431,-82.446872,5.0,17,"[Vegan, Organic Stores, Grocery]",Delivery available | Accepts credit cards | Lo...,"Small, but well stocked, with types of brands ...",[Rated 5.0 stars out of 5. Our neighbor texted...,1.263602,Black Radish Grocer is an all vegan grocer in ...,20.087016,"Small, but well stocked, with types of brands ...",0.357111,Where can I go for bbq and desserts and vegan ...
1,kGPCbjR2GiRE8Kdur6QTtA,The Vegan Halal Cart,,Tampa Bay,FL,33601,27.928239,-82.483403,5.0,12,"[Food Trucks, Vegan, Halal]",Delivery available | Outdoor seating available...,I would definitely give this truck a try if yo...,[Rated 5.0 stars out of 5. This place is so aw...,1.263602,The gyro is made from seitan and includes lett...,34.976046,The Vegan Halal Cart is overwhelmingly praised...,0.354034,Where can I go for bbq and desserts and vegan ...
2,JL6U8xzmWZJgVTZy6S4rsQ,Burgeezy,2442 Central Ave,Indianapolis,IN,46205,39.802471,-86.149930,5.0,16,"[Comfort Food, American (Traditional), Vegan, ...",Delivery available | Outdoor seating available...,I will definitely be returning to check out a ...,[Rated 5.0 stars out of 5. We love to see new ...,1.263602,The Crispy Chicken was the highlight. The BBQ ...,19.445342,Burgeezy is a highly praised vegan restaurant ...,2.185917,Where can I go for bbq and desserts and vegan ...
3,zsl17L34-EmgXbQushhfOQ,Smoove's Indy,10202 East Washington St,Indianapolis,IN,46229,39.779389,-85.986320,5.0,5,[Vegan],Delivery available | Take out available | Cate...,The menu does change occasionally to keep an e...,[Rated 5.0 stars out of 5. Smoove's Makes me s...,1.263602,Smoove's is a great vegan food option in Indy....,9.102459,Smoove's in Indy receives overwhelmingly posit...,2.251006,Where can I go for bbq and desserts and vegan ...
4,VGU-5m_0iR9pFriKyOD54g,The Sweet & Savory Kitchen,613 E Delano St,Tucson,AZ,85705,32.263307,-110.961980,5.0,5,"[Soul Food, Vegetarian, Vegan, American (New),...",Delivery available | Accepts credit cards | Bi...,The sizes were massive so you get more bang fo...,[Rated 5.0 stars out of 5. The food here is ou...,1.263602,The food here is out of this world!! We tried ...,8.864543,Customers rave about the flavorful and well-se...,1.666199,Where can I go for bbq and desserts and vegan ...
5,NUExM8_IQMh1csfeWs3PyQ,Hi Kori,1016 Race St,Philadelphia,PA,19107,39.955270,-75.156624,4.5,50,"[Bars, Japanese, Nightlife]",Delivery available | Accepts credit cards | St...,Only two of us were drinking but they demanded...,"[Rated 5.0 stars out of 5. Great cocktails, gr...",1.553797,Hi Kori infuses their liquor with tea. The dri...,26.385639,Hi Kori is a popular spot tucked away in China...,2.458389,I'm looking for a place that serves chinese an...
6,zgdGeg4L13C2Rbm4UARtHw,Island Sushi & Ramen,8716 W Fairview Ave,Boise,ID,83704,43.619719,-116.290423,4.5,313,"[Desserts, Japanese, Ramen, Sushi Bars]",Accepts credit cards | Lot parking available |...,I had the pork belly ramen lunch special with ...,[Rated 2.0 stars out of 5. I was really let do...,1.553797,Island Sushi & Ramen is one of the best ramen/...,70.586600,Customer reviews for Island Sushi and Ramen ar...,2.026218,I'm looking for a place that serves chinese an...
7,VVC2ZRQlz8JtK9LmgjQoIw,Sushi Thai by Samran,229 N Illinois St,Belleville,IL,62220,38.516467,-89.984443,4.5,53,"[Sushi Bars, Thai]",Accepts credit cards | Street parking availabl...,We had no trouble finding parking on a side st...,[Rated 5.0 stars out of 5. My wife and I went ...,1.553797,Sushi Thai is the best and ONLY sushi in town....,21.551154,Sushi Thai is a beloved local spot praised for...,1.958396,I'm looking for a place that serves chinese an...
8,TTCEDFbfYUec-xObcof88w,Van Loc,10648 98 Street NW,Edmonton,AB,T5H 2N7,53.5

In [13]:
with open("../summary_cache.json", "w", encoding="utf-8") as f:
    json.dump(olivia.get_summaries(), f, indent=2, ensure_ascii=False)

In [18]:
results.to_csv("cualitative_testing.csv", index=False)

In [15]:
time_cols = ["sumy_time", "bart_time", "gemini_time"]
results[time_cols].mean()

sumy_time       1.392395
bart_time      36.523287
gemini_time     1.620732
dtype: float64